# Input Data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df= pd.read_excel('/content/drive/My Drive/dataset/Skripsi/data_v3.xlsx') 
df.head()

,Tweet,Anotator 1 (Trifebi Shina),Anotator 2 (Mujaddid),Anotator 3 (Harris Ishaq),Sentimen Final
0,Alat penjajah sekarang adalah UU cipta kerja >...,Negatif,0,Negatif,Negatif
1,Aroma kepentingan asing dan pemilik modal sang...,Negatif,0,Negatif,Negatif
2,"Serentak di seluruh Nusantara, buruh menolak U...",Negatif,0,Negatif,Negatif
3,"Menjajah bangsa sendiri, usulan pemerintah zal...",Negatif,0,Negatif,Negatif
4,Negeri ini belum merdeka ..uu perbudakan lahir...,Negatif,0,Negatif,Negatif


In [ ]:
df.drop(['Anotator 1 (Trifebi Shina)', 'Anotator 2 (Mujaddid)', 'Anotator 3 (Harris Ishaq)'], axis=1, inplace=True)
df.rename(columns = {'Sentimen Final':'Sentimen'}, inplace = True)
df.head()

,Tweet,Sentimen
0,Alat penjajah sekarang adalah UU cipta kerja >...,Negatif
1,Aroma kepentingan asing dan pemilik modal sang...,Negatif
2,"Serentak di seluruh Nusantara, buruh menolak U...",Negatif
3,"Menjajah bangsa sendiri, usulan pemerintah zal...",Negatif
4,Negeri ini belum merdeka ..uu perbudakan lahir...,Negatif


In [ ]:
df.isnull().sum()

Tweet       0
Sentimen    0
dtype: int64

In [ ]:
df.shape

(1000, 2)

# Preprocessing

In [ ]:
pip install Sastrawi

     |████████████████████████████████| 215kB 8.3MB/s 


In [ ]:
#Import library untuk proses preprocessing
pd.set_option('display.max_colwidth', None)
import re
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from nltk.tokenize import word_tokenize 

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
#Case folding
df['Tweet'] = df['Tweet'].str.lower()
df.head(5)

,Tweet,Sentimen
0,alat penjajah sekarang adalah uu cipta kerja >> #buruhmenggugatuuciptakerja << #tolakuuciptakerja >> pic.twitter.com/wpdhlo3djq,Negatif
1,aroma kepentingan asing dan pemilik modal sangat terasa dalam uu cipta kerja. tolak uu cipta kerja >> #buruhmenggugatuuciptakerja << #uuciptakerjaalatpenjajahan#tolakuuciptakerja >> pic.twitter.com/i0p15fg9ba,Negatif
2,"serentak di seluruh nusantara, buruh menolak uu yang baru di sahkan yang meresahkan pekerja >> #buruhmenggugatuuciptakerja << #tolakuuciptakerja >> pic.twitter.com/whlraa20ck",Negatif
3,"menjajah bangsa sendiri, usulan pemerintah zalim dan disetujui dewan pengkhianat rakyat >> #buruhmenggugatuuciptakerja << #tolakuuciptakerja >> pic.twitter.com/zqlu4ej483",Negatif
4,negeri ini belum merdeka ..uu perbudakan lahir kembali oleh pemerintah zalim >> #buruhmenggugatuuciptakerja << #tolakuuciptakerja >> pic.twitter.com/zgcixpzqdh,Negatif


In [ ]:
#Cleansing
def cleansing(docs):
  tweet = re.sub(r'(http\S+|https\S+|twitter.com\S+|pic.twitter\S+)', '', docs)
  clean_tweet = tweet.replace('\n', '')
  clean_tweet = re.sub('(@[A-Za-z0-9]+)|(#[A-Za-z0-9]+)|([^A-Za-z \t])|(\w+:\/\/\S+)','', clean_tweet)
  return clean_tweet

In [ ]:
df['Tweet'] = df['Tweet'].apply(cleansing)
df.head()

,Tweet,Sentimen
0,alat penjajah sekarang adalah uu cipta kerja,Negatif
1,aroma kepentingan asing dan pemilik modal sangat terasa dalam uu cipta kerja tolak uu cipta kerja,Negatif
2,serentak di seluruh nusantara buruh menolak uu yang baru di sahkan yang meresahkan pekerja,Negatif
3,menjajah bangsa sendiri usulan pemerintah zalim dan disetujui dewan pengkhianat rakyat,Negatif
4,negeri ini belum merdeka uu perbudakan lahir kembali oleh pemerintah zalim,Negatif


In [ ]:
#Tokenization
def tokenize(docs):
    return word_tokenize(docs)

In [ ]:
df['Token Tweet'] = df['Tweet'].apply(tokenize)
df.head()

,Tweet,Sentimen,Token Tweet
0,alat penjajah sekarang adalah uu cipta kerja,Negatif,"[alat, penjajah, sekarang, adalah, uu, cipta, kerja]"
1,aroma kepentingan asing dan pemilik modal sangat terasa dalam uu cipta kerja tolak uu cipta kerja,Negatif,"[aroma, kepentingan, asing, dan, pemilik, modal, sangat, terasa, dalam, uu, cipta, kerja, tolak, uu, cipta, kerja]"
2,serentak di seluruh nusantara buruh menolak uu yang baru di sahkan yang meresahkan pekerja,Negatif,"[serentak, di, seluruh, nusantara, buruh, menolak, uu, yang, baru, di, sahkan, yang, meresahkan, pekerja]"
3,menjajah bangsa sendiri usulan pemerintah zalim dan disetujui dewan pengkhianat rakyat,Negatif,"[menjajah, bangsa, sendiri, usulan, pemerintah, zalim, dan, disetujui, dewan, pengkhianat, rakyat]"
4,negeri ini belum merdeka uu perbudakan lahir kembali oleh pemerintah zalim,Negatif,"[negeri, ini, belum, merdeka, uu, perbudakan, lahir, kembali, oleh, pemerintah, zalim]"


In [ ]:
#Stopword removal
def stop_removal(docs):
  words = stopwords.words('indonesian')
  words.extend(['amp', 'sih', 'nih', 'nya', 'n', 'nn', 't', 'hehe', 'nan', 'loh', '&amp']) #Nanti ditambahin juga yang remove itu, karena ada beberapa kata penting dlm list stopword
  words.remove('tidak')
  words = set(words)
  return [word for word in docs if word not in words]

In [ ]:
df['Token Tweet'] = df['Token Tweet'].apply(stop_removal)
df.head()

,Tweet,Sentimen,Token Tweet
0,alat penjajah sekarang adalah uu cipta kerja,Negatif,"[alat, penjajah, uu, cipta, kerja]"
1,aroma kepentingan asing dan pemilik modal sangat terasa dalam uu cipta kerja tolak uu cipta kerja,Negatif,"[aroma, kepentingan, asing, pemilik, modal, uu, cipta, kerja, tolak, uu, cipta, kerja]"
2,serentak di seluruh nusantara buruh menolak uu yang baru di sahkan yang meresahkan pekerja,Negatif,"[serentak, nusantara, buruh, menolak, uu, sahkan, meresahkan, pekerja]"
3,menjajah bangsa sendiri usulan pemerintah zalim dan disetujui dewan pengkhianat rakyat,Negatif,"[menjajah, bangsa, usulan, pemerintah, zalim, disetujui, dewan, pengkhianat, rakyat]"
4,negeri ini belum merdeka uu perbudakan lahir kembali oleh pemerintah zalim,Negatif,"[negeri, merdeka, uu, perbudakan, lahir, pemerintah, zalim]"


In [ ]:
#Stemming
factory = StemmerFactory()
stemmer = factory.create_stemmer()

def stemming(docs):
  return [stemmer.stem(word) for word in docs]

In [ ]:
df['Token Tweet'] = df['Token Tweet'].apply(stemming)
df.head()

,Tweet,Sentimen,Token Tweet
0,alat penjajah sekarang adalah uu cipta kerja,Negatif,"[alat, jajah, uu, cipta, kerja]"
1,aroma kepentingan asing dan pemilik modal sangat terasa dalam uu cipta kerja tolak uu cipta kerja,Negatif,"[aroma, penting, asing, milik, modal, uu, cipta, kerja, tolak, uu, cipta, kerja]"
2,serentak di seluruh nusantara buruh menolak uu yang baru di sahkan yang meresahkan pekerja,Negatif,"[serentak, nusantara, buruh, tolak, uu, sahkan, resah, kerja]"
3,menjajah bangsa sendiri usulan pemerintah zalim dan disetujui dewan pengkhianat rakyat,Negatif,"[jajah, bangsa, usul, perintah, zalim, tuju, dewan, khianat, rakyat]"
4,negeri ini belum merdeka uu perbudakan lahir kembali oleh pemerintah zalim,Negatif,"[negeri, merdeka, uu, budak, lahir, perintah, zalim]"


# TF-IDF

In [ ]:
def join_text_list(texts):
    return ' '.join([text for text in texts])
df["Final Tweet"] = df["Token Tweet"].apply(join_text_list)
df.head()

,Tweet,Sentimen,Token Tweet,Final Tweet
0,alat penjajah sekarang adalah uu cipta kerja,Negatif,"[alat, jajah, uu, cipta, kerja]",alat jajah uu cipta kerja
1,aroma kepentingan asing dan pemilik modal sangat terasa dalam uu cipta kerja tolak uu cipta kerja,Negatif,"[aroma, penting, asing, milik, modal, uu, cipta, kerja, tolak, uu, cipta, kerja]",aroma penting asing milik modal uu cipta kerja tolak uu cipta kerja
2,serentak di seluruh nusantara buruh menolak uu yang baru di sahkan yang meresahkan pekerja,Negatif,"[serentak, nusantara, buruh, tolak, uu, sahkan, resah, kerja]",serentak nusantara buruh tolak uu sahkan resah kerja
3,menjajah bangsa sendiri usulan pemerintah zalim dan disetujui dewan pengkhianat rakyat,Negatif,"[jajah, bangsa, usul, perintah, zalim, tuju, dewan, khianat, rakyat]",jajah bangsa usul perintah zalim tuju dewan khianat rakyat
4,negeri ini belum merdeka uu perbudakan lahir kembali oleh pemerintah zalim,Negatif,"[negeri, merdeka, uu, budak, lahir, perintah, zalim]",negeri merdeka uu budak lahir perintah zalim


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer 

tf_idf = TfidfVectorizer(binary=True)
tfidf_mat = tf_idf.fit_transform(df["Final Tweet"]).toarray()

In [ ]:
tfidf = pd.DataFrame(tfidf_mat, columns=tf_idf.get_feature_names())
tfidf.head()

,aamiin,abai,abdi,abie,abk,absurd,acu,ada,adab,adart,adat,adek,adidaya,adil,administrasi,adu,agal,agama,agamawan,agenda,agraria,agung,ah,ahli,air,airlangga,aja,ajak,ajar,aju,akademik,akademisi,akal,akan,akhir,akhirat,akhlak,akibat,akomodasi,akselerasi,...,wakil,waktu,walkie,wapres,waras,warga,waris,warna,waspada,well,wenang,widodo,wikipedia,wilayah,wkwk,wna,woi,wow,woy,wuhan,wujud,xcxajc,xfxfxaxa,xfxfxxaexfxfxxan,ya,yah,yakali,yakin,yangg,yes,yg,yng,yra,yth,yuana,yudicial,yuk,zalim,zero,zqn
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.292791,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.347168,0.0,0.0


In [ ]:
X = tfidf.values
X

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [ ]:
label = np.where(df['Sentimen']=='Positif', 1, 0)
y = label
y[45:75]

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0])

# Klasifikasi SVM Parameter Default (Tanpa PSO) - Skenario 1

In [ ]:
from sklearn import model_selection, svm
from sklearn.model_selection import KFold

from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import LabelEncoder

from scipy.sparse import csr_matrix as matrixTransform

In [ ]:
class myClass:
    def __init__(self):
        self.recall = 0
        self.acc = 0
        self.pres = 0
    def getRecall(self):
        return self.recall
    def getPres(self):
        return self.pres
    def setRecall(self, recall):
        self.recall = recall
    def setPres(self, pres):
        self.pres = pres

In [ ]:
def classify2():

    SVM = svm.SVC(C=1, gamma='scale', kernel='rbf', probability=True)
    SVM.fit(Train_X_final, y_train)
    predictions_SVM = SVM.predict(Test_X_final)

    acc = accuracy_score(predictions_SVM, y_test)
    CM = confusion_matrix(y_test, predictions_SVM)
    TN = CM[0][0]
    FN = CM[1][0]
    TP = CM[1][1]
    FP = CM[0][1]
    try :
        pres = TP/(TP+FP)
    except :
        pres = 0  
    accu = (TP+TN)/(TP+FP+TN+FN)
    recall = TP/(TP+FN)

    acc = accuracy_score(y_test, predictions_SVM)
    recll = recall_score(y_test, predictions_SVM)
    prec = precision_score(y_test, predictions_SVM)
    
    obj.setRecall(recall)
    obj.setPres(pres)
    
    print("Confusion Matrix")
    print(CM)
    return accu

In [ ]:
obj = myClass()
acc_scores = []
presisi_scores = []
recall_scores = []
cv = KFold(n_splits=10, random_state=32, shuffle=True)
for train_index, test_index in cv.split(X):
    X_train, X_test, y_train, y_test = X[train_index], X[test_index], y[train_index], y[test_index]

    Test_X_final = matrixTransform(X_test)
    Train_X_final = matrixTransform(X_train)    
    print("Data Train :", len(X_train))
    print("Data Test :", len(X_test))

    acc= classify2()
    acc_scores.append(acc)
    presisi_scores.append(obj.getPres())
    recall_scores.append(obj.getRecall())

Data Train : 900
Data Test : 100
Confusion Matrix
[[44  4]
 [10 42]]
Data Train : 900
Data Test : 100
Confusion Matrix
[[43  2]
 [ 2 53]]
Data Train : 900
Data Test : 100
Confusion Matrix
[[44  1]
 [10 45]]
Data Train : 900
Data Test : 100
Confusion Matrix
[[55  0]
 [ 2 43]]
Data Train : 900
Data Test : 100
Confusion Matrix
[[46  1]
 [ 4 49]]
Data Train : 900
Data Test : 100
Confusion Matrix
[[44  0]
 [ 4 52]]
Data Train : 900
Data Test : 100
Confusion Matrix
[[56  3]
 [ 4 37]]
Data Train : 900
Data Test : 100
Confusion Matrix
[[53  1]
 [ 6 40]]
Data Train : 900
Data Test : 100
Confusion Matrix
[[51  2]
 [ 9 38]]
Data Train : 900
Data Test : 100
Confusion Matrix
[[49  1]
 [ 4 46]]


In [ ]:
print("Pengujian tanpa optimasi...")
print("Accuracy :", acc_scores)
print("Presisi :", presisi_scores)
print("Recall :", recall_scores)
print("Accuracy Average :", np.mean(acc_scores))
print("Presisi Average :", np.mean(presisi_scores)) 
print("Recall Average :", np.mean(recall_scores))

Pengujian tanpa optimasi...
Accuracy : [0.86, 0.96, 0.89, 0.98, 0.95, 0.96, 0.93, 0.93, 0.89, 0.95]
Presisi : [0.9130434782608695, 0.9636363636363636, 0.9782608695652174, 1.0, 0.98, 1.0, 0.925, 0.975609756097561, 0.95, 0.9787234042553191]
Recall : [0.8076923076923077, 0.9636363636363636, 0.8181818181818182, 0.9555555555555556, 0.9245283018867925, 0.9285714285714286, 0.9024390243902439, 0.8695652173913043, 0.8085106382978723, 0.92]
Accuracy Average : 0.9299999999999999
Presisi Average : 0.9664273871815331
Recall Average : 0.8898680655603688
